In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

spark = SparkSession.builder.master("local[1]") \
                    .appName('PysparkDemo') \
                    .getOrCreate()



In [0]:
data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False) 

In [0]:
from pyspark.sql.functions import col,struct,when
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)


updatedDF = df2.withColumn("OtherInfo", 
    struct(col("id").alias("identifier"),
    col("gender").alias("gender"),
    col("salary").alias("salary"),
    when(col("salary").cast(IntegerType()) < 2000,"Low")
      .when(col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
-- name: struct (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

+--------------------+-----+------+------+
name |id |gender|salary|
+--------------------+-----+------+------+
[James, , Smith] |36636|M |3100 |
[Michael, Rose, ] |40288|M |4300 |
[Robert, , Williams]|42114|M |1400 |
[Maria, Anne, Jones]|39192|F |5500 |
[Jen, Mary, Brown] | |F |-1 |
+--------------------+-----+------+------+

root
-- name: struct (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
-- OtherInfo: struct (nullable = false)
 |-- identifier: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
name |OtherInfo |
+--------------------+------------------------+
[James, , Smith] |[36636, M, 3100, Medium]|
[Michael, Rose, ] |[40288, M, 4300, High] |
[Robert, , Williams]|[42114, M, 1400, Low] |
[Maria, Anne, Jones]|[39192, F, 5500, High] |
[Jen, Mary, Brown] |[, F, -1, Low] |
+--------------------+------------------------+

In [0]:
df=spark.read.option("header","True").option("inferSchema","True").option("sep",",").csv("/FileStore/tables/diabetes-2.csv")
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
 6| 148| 72| 35| 0|33.6| 0.627| 50| 1|
 1| 85| 66| 29| 0|26.6| 0.351| 31| 0|
 8| 183| 64| 0| 0|23.3| 0.672| 32| 1|
 1| 89| 66| 23| 94|28.1| 0.167| 21| 0|
 0| 137| 40| 35| 168|43.1| 2.288| 33| 1|
 5| 116| 74| 0| 0|25.6| 0.201| 30| 0|
 3| 78| 50| 32| 88|31.0| 0.248| 26| 1|
 10| 115| 0| 0| 0|35.3| 0.134| 29| 0|
 2| 197| 70| 45| 543|30.5| 0.158| 53| 1|
 8| 125| 96| 0| 0| 0.0| 0.232| 54| 1|
 4| 110| 92| 0| 0|37.6| 0.191| 30| 0|
 10| 168| 74| 0| 0|38.0| 0.537| 34| 1|
 10| 139| 80| 0| 0|27.1| 1.441| 57| 0|
 1| 189| 60| 23| 846|30.1| 0.398| 59| 1|
 5| 166| 72| 19| 175|25.8| 0.587| 51| 1|
 7| 100| 0| 0| 0|30.0| 0.484| 32| 1|
 0| 118| 84| 47| 230|45.8| 0.551| 31| 1|
 7| 107| 74| 0| 0|29.6| 0.254| 31| 1|
 1| 103| 30| 38| 83|43.3| 0.183| 33| 0|
 1| 115| 70| 30| 96|34.6| 0.529| 32| 1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 20 rows

In [0]:

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import * 
import pyspark.sql.functions as F
from pyspark.sql.functions import col, asc,desc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler,StandardScaler
from pyspark.ml import Pipeline
from sklearn.metrics import confusion_matrix


In [0]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Pregnancies,768,3.8450520833333335,3.36957806269887,0,17
Glucose,768,120.89453125,31.97261819513622,0,199
BloodPressure,768,69.10546875,19.355807170644777,0,122
SkinThickness,768,20.536458333333332,15.952217567727642,0,99
Insulin,768,79.79947916666667,115.24400235133803,0,846
Age,768,33.240885416666664,11.760231540678689,21,81
Outcome,768,0.3489583333333333,0.476951377242799,0,1


In [0]:
from matplotlib import cm
fig = plt.figure(figsize=(25,15)) ## Plot Size 
st = fig.suptitle("Distribution of Features", fontsize=50,
                  verticalalignment='center') # Plot Main Title 

for col,num in zip(df.toPandas().describe().columns, range(1,11)):
    ax = fig.add_subplot(3,4,num)
    ax.hist(df.toPandas()[col])
    plt.style.use('dark_background') 
    plt.grid(False)
    plt.xticks(rotation=45,fontsize=20)
    plt.yticks(fontsize=15)
    plt.title(col.upper(),fontsize=20)
plt.tight_layout()
st.set_y(0.95)
fig.subplots_adjust(top=0.85,hspace = 0.4)
plt.show()

In [0]:

from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,0,0,0,0,0,0,0,0


In [0]:

from pyspark.sql.functions import udf
y_udf = udf(lambda y: "No" if y==0 else "yes", StringType())

df=df.withColumn("HasDiabities", y_udf('OutCome')).drop("OutCome")

In [0]:
def udf_multiple(age):
      if (age <= 25):
        return 'Under 25'
      elif (age >= 25 and age <= 35):
        return 'Between 25 and 35'
      elif (age > 35 and age < 50):
        return 'Between 36 and 49'
      elif (age >= 50):
        return 'Over 50'
      else: return 'N/A'

education_udf = udf(udf_multiple)
df=df.withColumn("Age_udf", education_udf('Age'))

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import rank,sum,col
from pyspark.sql import Window

window = Window.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
tab = df.select(['Age_udf','Glucose']).\
   groupBy('Age_udf').\
       agg(F.count('Glucose').alias('UserCount'),
           F.mean('Glucose').alias('Glucose_AVG'),
           F.min('Glucose').alias('Glucose_MIN'),
           F.max('Glucose').alias('Glucose_MAX')).\
       withColumn('total',sum(col('UserCount')).over(window)).\
       withColumn('Percent',col('UserCount')*100/col('total')).\
       drop(col('total')).sort(desc("Percent"))

tab.show()

+-----------------+---------+------------------+-----------+-----------+------------------+
 Age_udf|UserCount| Glucose_AVG|Glucose_MIN|Glucose_MAX| Percent|
+-----------------+---------+------------------+-----------+-----------+------------------+
 Under 25| 267|110.71910112359551| 0| 199| 34.765625|
Between 25 and 35| 231|121.67099567099567| 71| 198| 30.078125|
Between 36 and 49| 181|125.74033149171271| 0| 197|23.567708333333332|
 Over 50| 89| 139.5505617977528| 57| 197|11.588541666666666|
+-----------------+---------+------------------+-----------+-----------+------------------+

In [0]:
numeric_features = [t[0] for t in df.dtypes if t[1] != 'string']
numeric_features_df=df.select(numeric_features)
numeric_features_df.toPandas().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
spark.sql("""SELECT 
explode(
    from_json(
        concat('{"data":', 
               '[{"a": 1.0,"b": 1},{"a": 0.0,"b": 2}]', 
               '}'), 
        'data array<struct<a:DOUBLE, b:INT>>'
    ).data) as data""").show()

+--------+
 data|
+--------+
[1.0, 1]|
[0.0, 2]|
+--------+

In [0]:
from pyspark.sql import Row
jstr1 = u'{"header":{"id":12345,"foo":"bar"},"body":{"id":111000,"name":"foobar","sub_json":{"id":54321,"sub_sub_json":{"col1":20,"col2":"somethong"}}}}'
jstr2 = u'{"header":{"id":12346,"foo":"baz"},"body":{"id":111002,"name":"barfoo","sub_json":{"id":23456,"sub_sub_json":{"col1":30,"col2":"something else"}}}}'
jstr3 = u'{"header":{"id":43256,"foo":"foobaz"},"body":{"id":20192,"name":"bazbar","sub_json":{"id":39283,"sub_sub_json":{"col1":50,"col2":"another thing"}}}}'
df = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)])

In [0]:
df.show()

+--------------------+
 json|
+--------------------+
{"header":{"id":1...|
{"header":{"id":1...|
{"header":{"id":4...|
+--------------------+

In [0]:
from pyspark.sql.functions import from_json, col
json_schema = spark.read.json(df.rdd.map(lambda row: row.json)).schema
df.withColumn('json', from_json(col('json'), json_schema)).show()

+--------------------+
 json|
+--------------------+
[[111000, foobar,...|
[[111002, barfoo,...|
[[20192, bazbar, ...|
+--------------------+

In [0]:
df.withColumn('json', from_json(col('json'), json_schema)).select(col('json.body.id').alias('body_id')).show()

+-------+
body_id|
+-------+
 111000|
 111002|
 20192|
+-------+

In [0]:
pip install --upgrade tensorflow

Python interpreter will be restarted.
Collecting tensorflow
 Downloading tensorflow-2.4.0-cp37-cp37m-manylinux2010_x86_64.whl (394.7 MB)
Collecting gast==0.3.3
 Downloading gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
Collecting absl-py~=0.10
 Downloading absl_py-0.11.0-py3-none-any.whl (127 kB)
Collecting astunparse~=1.6.3
 Downloading astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Collecting flatbuffers~=1.12.0
 Downloading flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
Collecting google-pasta~=0.2
 Downloading google_pasta-0.2.0-py3-none-any.whl (57 kB)
Collecting grpcio~=1.32.0
 Downloading grpcio-1.32.0-cp37-cp37m-manylinux2014_x86_64.whl (3.8 MB)
Collecting h5py~=2.10.0
 Downloading h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
Collecting keras-preprocessing~=1.1.2
 Downloading Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Collecting numpy~=1.19.2
 Downloading numpy-1.19.4-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
Collecting opt-einsum~=3.3.0
 Downloading opt_einsum-3.3.0-py3-none-any.whl (65 kB)
Collecting protobuf>=3.9.2
 Downloading protobuf-3.14.0-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
Collecting six~=1.15.0
 Downloading six-1.15.0-py2.py3-none-any.whl (10 kB)
Collecting tensorboard~=2.4
 Downloading tensorboard-2.4.0-py3-none-any.whl (10.6 MB)
Requirement already satisfied: setuptools>=41.0.0 in /usr/local/lib/python3.7/dist-packages (from tensorboard~=2.4->tensorflow) (45.2.0)
Requirement already satisfied: requests<3,>=2.21.0 in /databricks/python3/lib/python3.7/site-packages (from tensorboard~=2.4->tensorflow) (2.22.0)
Collecting google-auth<2,>=1.6.3
 Downloading google_auth-1.24.0-py2.py3-none-any.whl (114 kB)
Collecting cachetools<5.0,>=2.0.0
 Downloading cachetools-4.2.0-py3-none-any.whl (12 kB)
Collecting google-auth-oauthlib<0.5,>=0.4.1
 Downloading google_auth_oauthlib-0.4.2-py2.py3-none-any.whl (18 kB)
Collecting markdown>=2.6.8
 Downloading Markdown-3.3.3-py3-none-any.whl (96 kB)
Collecting pyasn1-modules>=0.2.1
 Downloading pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
Collecting pyasn1<0.5.0,>=0.4.6
 Downloading pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow) (2020.6.20)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow) (2.8)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow) (1.25.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow) (3.0.4)
Collecting requests-oauthlib>=0.7.0
 Downloading requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
Collecting oauthlib>=3.0.0
 Downloading oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
Collecting rsa<5,>=3.1.4
 Downloading rsa-4.6-py3-none-any.whl (47 kB)
Collecting tensorboard-plugin-wit>=1.6.0
 Downloading tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
Collecting tensorflow-estimator<2.5.0,>=2.4.0rc0
 Downloading tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
Collecting termcolor~=1.1.0
 Downloading termcolor-1.1.0.tar.gz (3.9 kB)
Collecting typing-extensions~=3.7.4
 Downloading typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
Collecting werkzeug>=0.11.15
 Downloading Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
Collecting wheel~=0.35
 Downloading wheel-0.36.2-py2.py3-none-any.whl (35 kB)
Collecting wrapt~=1.12.1
 Downloading wrapt-1.12.1.tar.gz (27 kB)
Collecting importlib-metadata
 Downloading importlib_metadata-3.3.0-py3-none-any.whl (10 kB)
Collecting zipp>=0.5
 Downloading zipp-3.4.0-py3-none-any.whl (5.2 kB)
Building wheels for collected packages: termcolor, wrapt
 Building wheel for termcolor (setup.py): started
 Building wheel for termcolor (

In [0]:
import tensorflow as tf
a = tf.Variable(0)
print(tf)

<module 'tensorflow' from '/local_disk0/.ephemeral_nfs/envs/pythonEnv-2a2ff172-2a5b-4c06-918b-ea872454f537/lib/python3.7/site-packages/tensorflow/__init__.py'>